## <center> Packages & Connection </center>

In [2]:
import os
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [4]:
## Connection
connection = presto.connect(
        host='presto-gateway.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='manoj.ravirajan@rapido.bike'
)

## <center> Local extracted files </center>

In [5]:
notebook_path = os.getcwd()
print(notebook_path)

/Users/rapido/code-repository/pricing/affluence/ocara


In [6]:
refined_hex_8_all_data = pd.read_csv('/Users/rapido/local-datasets/affluence/processed/hex_8_all_data_2023_07_10_to_16.csv')
refined_hex_8_hsr_yesh = pd.read_csv('/Users/rapido/local-datasets/affluence/processed/hex_8_data_2023_07_10_to_16.csv')

In [7]:
print(refined_hex_8_all_data.shape)
print(refined_hex_8_hsr_yesh.shape)

(4032, 29)
(30, 29)


In [8]:
hex_8_all_data = refined_hex_8_all_data[refined_hex_8_all_data['service_name'].isin(['Link'])]
hex_8_all_data.describe()

,fe_cus_count,fe_count,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders,aor,fe2rr,fe2net,taxi_high_income,taxi_medium_income,taxi_low_income,link_only_service,auto_only_service,both_service,link_ps,link_nps,auto_ps,auto_nps,fe_intent_stable,fe_intent_increasing,fe_intent_declining
count,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1998.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,1025.482741,3759.492246,675.464232,2.057529,40.968984,131.326663,502.478739,111.851426,271.029515,62.354354,18.061531,5.417709,404.630815,272.687844,51.189095,98.815408,144.536268,645.574287,139.452726,245.380190,221.260130,363.391696,337.763882,403.431216,170.871436
std,2103.330224,8212.575206,1432.721151,3.915677,120.463644,314.321455,1026.933039,232.362412,557.164357,29.021819,8.953372,3.944067,894.232561,537.595619,96.525761,189.521964,292.914341,1383.512775,281.119674,520.950135,486.036235,773.491357,678.963273,848.749936,377.831765
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,62.500000,8.000000,0.000000,1.000000,1.000000,4.000000,1.000000,1.000000,50.000000,14.000000,2.000000,5.000000,5.000000,1.000000,3.000000,2.000000,8.000000,2.000000,3.000000,2.000000,4.000000,5.000000,6.000000,2.000000
50%,143.000000,453.000000,78.000000,1.000000,5.000000,13.000000,57.000000,16.000000,26.000000,73.000000,18.000000,6.000000,43.000000,39.000000,8.000000,19.000000,18.000000,70.000000,18.000000,28.000000,21.000000,40.000000,43.000000,49.000000,18.000000
75%,1090.500000,3643.000000,675.500000,3.000000,21.000000,101.000000,537.000000,118.000000,287.000000,82.000000,21.000000,8.000000,391.000000,308.500000,61.000000,118.000000,152.500000,655.500000,151.000000,244.500000,211.000000,364.000000,367.500000,415.000000,161.000000
max,25922.000000,100666.000000,14122.000000,77.000000,1368.000000,2937.000000,10310.000000,2898.000000,5596.000000,100.000000,100.000000,50.000000,12122.000000,6695.000000,1232.000000,2966.000000,3669.000000,17808.000000,3152.000000,6463.000000,6398.000000,10067.000000,8480.000000,10943.000000,4981.000000


In [9]:
hex_8_all_data.columns

Index(['city', 'service_name', 'pickup_cluster', 'pickup_hex_8',
       'fe_cus_count', 'fe_count', 'requested_orders', 'cobrm',
       'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders',
       'aor', 'fe2rr', 'fe2net', 'taxi_high_income', 'taxi_medium_income',
       'taxi_low_income', 'link_only_service', 'auto_only_service',
       'both_service', 'link_ps', 'link_nps', 'auto_ps', 'auto_nps',
       'fe_intent_stable', 'fe_intent_increasing', 'fe_intent_declining'],
      dtype='object')

In [10]:
df_hex_8_all_data = hex_8_all_data[['city', 'service_name', 'pickup_cluster', 'pickup_hex_8',
       'fe_cus_count', 'fe_count', 'requested_orders', 'cobrm',
       'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders',
       'aor', 'fe2rr', 'fe2net', 'taxi_high_income', 'taxi_medium_income',
       'taxi_low_income', 'link_ps', 'link_nps', 'auto_ps', 'auto_nps']]

df_hex_8_all_data

,city,service_name,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders,aor,fe2rr,fe2net,taxi_high_income,taxi_medium_income,taxi_low_income,link_ps,link_nps,auto_ps,auto_nps
0,Bangalore,Link,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0,0,0,0,0.0,4.0,0.0,10,8,0,4,6,3,7
2,Bangalore,Link,Attibele,8861893713fffff,112,268,21,2,1,10,8,4,2,38.0,8.0,1.0,23,20,5,7,9,13,13
3,Bangalore,Link,Attibele,88618936a7fffff,4,22,2,0,0,2,0,0,0,0.0,9.0,0.0,1,1,0,0,0,0,1
5,Bangalore,Link,Sarjapur,88618935ebfffff,11,57,9,4,0,0,4,2,2,44.0,16.0,4.0,5,1,2,1,3,0,2
7,Bangalore,Link,Sarjapur,88618935e9fffff,1,1,1,0,1,0,1,0,0,100.0,100.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,Bangalore,Link,Bidadi,886014413dfffff,5,12,1,1,0,0,0,0,0,0.0,8.0,0.0,3,1,0,1,0,1,1
4021,Bangalore,Link,Bidadi,8860144135fffff,29,60,1,0,0,0,0,0,0,0.0,2.0,0.0,10,0,0,0,0,0,1
4022,Bangalore,Link,Bidadi,8860144125fffff,3,15,2,0,0,0,0,0,0,0.0,13.0,0.0,0,0,0,0,0,0,0
4024,Bangalore,Link,Bidadi,8860144113fffff,1,3,2,0,0,0,0,0,0,0.0,67.0,0.0,0,0,0,0,0,0,1


## <center> Adding new required columns </center>

In [11]:
df_hex_8_all_data['ocara_%'] = (df_hex_8_all_data['ocara']*100/df_hex_8_all_data['requested_orders']).round(2)
df_hex_8_all_data['high_income_%'] = (df_hex_8_all_data['taxi_high_income']*100/df_hex_8_all_data['fe_cus_count']).round(2)
df_hex_8_all_data['high_income_thrshld'] = df_hex_8_all_data['high_income_%'].median()

df_hex_8_all_data['affluence_hi_tag'] = np.where(
                                        df_hex_8_all_data['high_income_%'] <= df_hex_8_all_data['high_income_thrshld'] , 
                                        'Less', 
                                        'High')

In [12]:
df_hex_8_all_data.fe_count.describe()

count      1999.000000
mean       3759.492246
std        8212.575206
min           1.000000
25%          62.500000
50%         453.000000
75%        3643.000000
max      100666.000000
Name: fe_count, dtype: float64

In [13]:
df_hex_8_all_data['ocara_%'].describe()

count    1998.000000
mean       16.909139
std        14.148424
min         0.000000
25%        11.247500
50%        16.670000
75%        21.150000
max       100.000000
Name: ocara_%, dtype: float64

In [14]:
df_hex_8_all_data.head(5)

,city,service_name,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders,aor,fe2rr,fe2net,taxi_high_income,taxi_medium_income,taxi_low_income,link_ps,link_nps,auto_ps,auto_nps,ocara_%,high_income_%,high_income_thrshld,affluence_hi_tag
0,Bangalore,Link,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0,0,0,0,0.0,4.0,0.0,10,8,0,4,6,3,7,0.00,25.00,32.11,Less
2,Bangalore,Link,Attibele,8861893713fffff,112,268,21,2,1,10,8,4,2,38.0,8.0,1.0,23,20,5,7,9,13,13,19.05,20.54,32.11,Less
3,Bangalore,Link,Attibele,88618936a7fffff,4,22,2,0,0,2,0,0,0,0.0,9.0,0.0,1,1,0,0,0,0,1,0.00,25.00,32.11,Less
5,Bangalore,Link,Sarjapur,88618935ebfffff,11,57,9,4,0,0,4,2,2,44.0,16.0,4.0,5,1,2,1,3,0,2,22.22,45.45,32.11,High
7,Bangalore,Link,Sarjapur,88618935e9fffff,1,1,1,0,1,0,1,0,0,100.0,100.0,0.0,0,0,0,0,0,0,0,0.00,0.00,32.11,Less


In [15]:
## Filtering required columns

df_refined_data = df_hex_8_all_data [[
                                        'city', 'pickup_cluster', 'pickup_hex_8',
                                        'fe_cus_count', 'fe_count', 'requested_orders', 'net_orders', 
                                        'ocara', 'ocara_%',
                                        'high_income_%', 'high_income_thrshld', 'affluence_hi_tag'                                    ]]

In [16]:
df_refined_data

,city,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,net_orders,ocara,ocara_%,high_income_%,high_income_thrshld,affluence_hi_tag
0,Bangalore,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0.00,25.00,32.11,Less
2,Bangalore,Attibele,8861893713fffff,112,268,21,2,4,19.05,20.54,32.11,Less
3,Bangalore,Attibele,88618936a7fffff,4,22,2,0,0,0.00,25.00,32.11,Less
5,Bangalore,Sarjapur,88618935ebfffff,11,57,9,2,2,22.22,45.45,32.11,High
7,Bangalore,Sarjapur,88618935e9fffff,1,1,1,0,0,0.00,0.00,32.11,Less
...,...,...,...,...,...,...,...,...,...,...,...,...
4017,Bangalore,Bidadi,886014413dfffff,5,12,1,0,0,0.00,60.00,32.11,High
4021,Bangalore,Bidadi,8860144135fffff,29,60,1,0,0,0.00,34.48,32.11,High
4022,Bangalore,Bidadi,8860144125fffff,3,15,2,0,0,0.00,0.00,32.11,Less
4024,Bangalore,Bidadi,8860144113fffff,1,3,2,0,0,0.00,0.00,32.11,Less


In [17]:
df_refined_data.describe()

,fe_cus_count,fe_count,requested_orders,net_orders,ocara,ocara_%,high_income_%,high_income_thrshld
count,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1998.000000,1999.000000,1.999000e+03
mean,1025.482741,3759.492246,675.464232,271.029515,111.851426,16.909139,30.224972,3.211000e+01
std,2103.330224,8212.575206,1432.721151,557.164357,232.362412,14.148424,13.754678,4.903972e-13
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.211000e+01
25%,22.000000,62.500000,8.000000,1.000000,1.000000,11.247500,23.525000,3.211000e+01
50%,143.000000,453.000000,78.000000,26.000000,16.000000,16.670000,32.110000,3.211000e+01
75%,1090.500000,3643.000000,675.500000,287.000000,118.000000,21.150000,37.820000,3.211000e+01
max,25922.000000,100666.000000,14122.000000,5596.000000,2898.000000,100.000000,100.000000,3.211000e+01


## Analysis
      

In [18]:
df_analysis = df_refined_data
df_analysis.shape

(1999, 12)

In [19]:
df_analysis

,city,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,net_orders,ocara,ocara_%,high_income_%,high_income_thrshld,affluence_hi_tag
0,Bangalore,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0.00,25.00,32.11,Less
2,Bangalore,Attibele,8861893713fffff,112,268,21,2,4,19.05,20.54,32.11,Less
3,Bangalore,Attibele,88618936a7fffff,4,22,2,0,0,0.00,25.00,32.11,Less
5,Bangalore,Sarjapur,88618935ebfffff,11,57,9,2,2,22.22,45.45,32.11,High
7,Bangalore,Sarjapur,88618935e9fffff,1,1,1,0,0,0.00,0.00,32.11,Less
...,...,...,...,...,...,...,...,...,...,...,...,...
4017,Bangalore,Bidadi,886014413dfffff,5,12,1,0,0,0.00,60.00,32.11,High
4021,Bangalore,Bidadi,8860144135fffff,29,60,1,0,0,0.00,34.48,32.11,High
4022,Bangalore,Bidadi,8860144125fffff,3,15,2,0,0,0.00,0.00,32.11,Less
4024,Bangalore,Bidadi,8860144113fffff,1,3,2,0,0,0.00,0.00,32.11,Less


In [20]:
df_analysis[['fe_count','ocara_%']].describe()

,fe_count,ocara_%
count,1999.000000,1998.000000
mean,3759.492246,16.909139
std,8212.575206,14.148424
min,1.000000,0.000000
25%,62.500000,11.247500
50%,453.000000,16.670000
75%,3643.000000,21.150000
max,100666.000000,100.000000


## Flag

In [21]:
df_analysis['flag_fe_density'] = np.where(df_analysis['fe_count'] >= 3643, 'High fe', 'Less fe')
df_analysis['fe_bucket'] = \
        np.where(df_analysis['fe_count'] <= 453, '<=453',
        np.where(df_analysis['fe_count'] <= 3000, '<=3000',
        np.where(df_analysis['fe_count'] < 3643, '3000-3643','3643+')))
df_analysis


df_analysis['ocara_tag'] = \
        np.where(df_analysis['ocara_%'] <= 11, 'Q1',
        np.where(df_analysis['ocara_%'] <= 16, 'Q2',
        np.where(df_analysis['ocara_%'] <= 21 , 'Q3','Q4')))
df_analysis

df_analysis['ocara_tag_1'] = \
        np.where(df_analysis['ocara_%'] <= 16, 'less', 'high')
df_analysis

,city,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,net_orders,ocara,ocara_%,high_income_%,high_income_thrshld,affluence_hi_tag,flag_fe_density,fe_bucket,ocara_tag,ocara_tag_1
0,Bangalore,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0.00,25.00,32.11,Less,Less fe,<=453,Q1,less
2,Bangalore,Attibele,8861893713fffff,112,268,21,2,4,19.05,20.54,32.11,Less,Less fe,<=453,Q3,high
3,Bangalore,Attibele,88618936a7fffff,4,22,2,0,0,0.00,25.00,32.11,Less,Less fe,<=453,Q1,less
5,Bangalore,Sarjapur,88618935ebfffff,11,57,9,2,2,22.22,45.45,32.11,High,Less fe,<=453,Q4,high
7,Bangalore,Sarjapur,88618935e9fffff,1,1,1,0,0,0.00,0.00,32.11,Less,Less fe,<=453,Q1,less
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,Bangalore,Bidadi,886014413dfffff,5,12,1,0,0,0.00,60.00,32.11,High,Less fe,<=453,Q1,less
4021,Bangalore,Bidadi,8860144135fffff,29,60,1,0,0,0.00,34.48,32.11,High,Less fe,<=453,Q1,less
4022,Bangalore,Bidadi,8860144125fffff,3,15,2,0,0,0.00,0.00,32.11,Less,Less fe,<=453,Q1,less
4024,Bangalore,Bidadi,8860144113fffff,1,3,2,0,0,0.00,0.00,32.11,Less,Less fe,<=453,Q1,less


In [22]:
df_analysis_v1 = df_analysis.groupby(['affluence_hi_tag', 'flag_fe_density' , 'ocara_tag', 'ocara_tag_1']).agg({'pickup_hex_8' : 'count',
                                                         'fe_count' : 'sum',
                                                         'requested_orders' : 'sum',
                                                         'net_orders' : 'sum',                                                  
                                                         'ocara' : 'sum'                           
                                                        }).reset_index()
df_analysis_v1

,affluence_hi_tag,flag_fe_density,ocara_tag,ocara_tag_1,pickup_hex_8,fe_count,requested_orders,net_orders,ocara
0,High,High fe,Q1,less,2,11821,2704,1066,277
1,High,High fe,Q2,less,212,3145418,605249,241374,85730
2,High,High fe,Q3,high,224,2843726,475109,194219,84407
3,High,High fe,Q4,high,20,192835,32398,11193,7622
4,High,Less fe,Q1,less,135,12496,2192,708,158
5,High,Less fe,Q2,less,91,121676,23959,9672,3418
6,High,Less fe,Q3,high,173,313664,57732,24794,10532
7,High,Less fe,Q4,high,141,108653,19101,7549,4512
8,Less,High fe,Q2,less,5,31873,6539,2734,1018
9,Less,High fe,Q3,high,26,185657,32649,13474,6179


In [23]:
df_analysis_v1['hex_distribution'] = (df_analysis_v1['pickup_hex_8']*100/df_analysis_v1.pickup_hex_8.sum()).round(2)
df_analysis_v1['fe2rr'] = (df_analysis_v1['requested_orders']*100/df_analysis_v1['fe_count']).round(2)
df_analysis_v1['ocara_%'] = (df_analysis_v1['ocara']*100/df_analysis_v1['requested_orders']).round(2)

In [24]:
df_analysis_v1

,affluence_hi_tag,flag_fe_density,ocara_tag,ocara_tag_1,pickup_hex_8,fe_count,requested_orders,net_orders,ocara,hex_distribution,fe2rr,ocara_%
0,High,High fe,Q1,less,2,11821,2704,1066,277,0.10,22.87,10.24
1,High,High fe,Q2,less,212,3145418,605249,241374,85730,10.61,19.24,14.16
2,High,High fe,Q3,high,224,2843726,475109,194219,84407,11.21,16.71,17.77
3,High,High fe,Q4,high,20,192835,32398,11193,7622,1.00,16.80,23.53
4,High,Less fe,Q1,less,135,12496,2192,708,158,6.75,17.54,7.21
5,High,Less fe,Q2,less,91,121676,23959,9672,3418,4.55,19.69,14.27
6,High,Less fe,Q3,high,173,313664,57732,24794,10532,8.65,18.41,18.24
7,High,Less fe,Q4,high,141,108653,19101,7549,4512,7.05,17.58,23.62
8,Less,High fe,Q2,less,5,31873,6539,2734,1018,0.25,20.52,15.57
9,Less,High fe,Q3,high,26,185657,32649,13474,6179,1.30,17.59,18.93


In [25]:
df_analysis_v1.groupby(['affluence_hi_tag']).agg({'pickup_hex_8' : 'sum',
                                                    'hex_distribution': 'sum', 
                                                    'fe_count' : 'sum',
                                                    'requested_orders' : 'sum',
                                                    'ocara' : 'sum'
                                                   })

,pickup_hex_8,hex_distribution,fe_count,requested_orders,ocara
affluence_hi_tag,,,,,
High,998,49.92,6750289,1218444,196656
Less,1001,50.07,764936,131809,26935


In [26]:
print(196656*100.0/1218444)
print(26935*100.0/131809)

16.139929286860948
20.43487167037152


#### Interpretation

        -> ocara % higher in Less affluence group (4%)

In [92]:
df_analysis_v1.groupby(['flag_fe_density']).agg({'pickup_hex_8' : 'sum',
                                                    'hex_distribution': 'sum', 
                                                    'fe_count' : 'sum',
                                                    'requested_orders' : 'sum',
                                                    'ocara' : 'sum'
                                                   })

,pickup_hex_8,hex_distribution,fe_count,requested_orders,ocara
flag_fe_density,,,,,
High fe,501,25.07,6587415,1181169,191491
Less fe,1498,74.92,927810,169084,32100


In [28]:
print(191491*100.0/1181169)
print(32100*100.0/169084)

16.211989986191647
18.984646684488183


#### Interpretation

        -> ocara % higher in Less FE density group (~2.5% less significant)

In [29]:
df_analysis_v1.groupby(['ocara_tag_1']).agg({'pickup_hex_8' : 'sum',
                                                    'hex_distribution': 'sum', 
                                                    'fe_count' : 'sum',
                                                    'requested_orders' : 'sum',
                                                    'ocara' : 'sum'
                                                   })

,pickup_hex_8,hex_distribution,fe_count,requested_orders,ocara
ocara_tag_1,,,,,
high,1112,55.62,4134859,700077,131941
less,887,44.37,3380366,650176,91650


In [30]:
print(131941*100.0/700077)
print(91650*100.0/650176)

18.846641155187214
14.096183187321587


Interpretation

        -> Based in ocara tag

In [31]:
df_analysis_v1.groupby(['ocara_tag']).agg({'pickup_hex_8' : 'sum',
                                                    'hex_distribution': 'sum', 
                                                    'fe_count' : 'sum',
                                                    'requested_orders' : 'sum',
                                                    'ocara' : 'sum'
                                                   })

,pickup_hex_8,hex_distribution,fe_count,requested_orders,ocara
ocara_tag,,,,,
Q1,484,24.21,46348,8260,618
Q2,403,20.16,3334018,641916,91032
Q3,600,30.01,3486378,592038,106087
Q4,512,25.61,648481,108039,25854


In [32]:
print(618*100.0/8260)
print(91032*100.0/641916)
print(106087*100.0/592038)
print(25854*100.0/108039)

7.481840193704601
14.181294748845643
17.918951148406016
23.930247410657262


In [33]:
df_v1 = df_analysis_v1.groupby(['affluence_hi_tag','flag_fe_density']).agg({'pickup_hex_8' : 'sum',
                                                                    'hex_distribution': 'sum', 
                                                                    'fe_count' : 'sum',
                                                                    'requested_orders' : 'sum',
                                                                    'ocara' : 'sum'
                                                                   }).reset_index()
df_v1['fe2rr'] = (df_v1['requested_orders']*100/df_v1['fe_count']).round(2)
df_v1['ocara_%'] = (df_v1['ocara']*100/df_v1['requested_orders']).round(2)

df_v1[['affluence_hi_tag','flag_fe_density','hex_distribution', 'pickup_hex_8', 'fe2rr','ocara_%']]


## Highlights 

## Affluence x FE Density

,affluence_hi_tag,flag_fe_density,hex_distribution,pickup_hex_8,fe2rr,ocara_%
0,High,High fe,22.92,458,18.01,15.96
1,High,Less fe,27.00,540,18.51,18.08
2,Less,High fe,2.15,43,16.69,20.48
3,Less,Less fe,47.92,958,17.80,20.39


Interpretation 

        -> Ocara % generally gets increased when fe density is lesser.
        
        -> Ocara % is high when it's identified less affluence hex.

In [34]:
df_v2 = df_analysis_v1.groupby(['affluence_hi_tag','ocara_tag_1']).agg({ 'pickup_hex_8' : 'sum',
                                                              'hex_distribution': 'sum', 
                                                                    'fe_count' : 'sum',
                                                                    'requested_orders' : 'sum',
                                                                    'ocara' : 'sum'
                                                                   }).reset_index()

df_v2['fe2rr'] = (df_v2['requested_orders']*100/df_v2['fe_count']).round(2)
df_v2['ocara_%'] = (df_v2['ocara']*100/df_v2['requested_orders']).round(2)

df_v2[['affluence_hi_tag','ocara_tag_1','hex_distribution','fe2rr','ocara_%']]

## Affluence x OCARA

,affluence_hi_tag,ocara_tag_1,hex_distribution,fe2rr,ocara_%
0,High,high,27.91,16.89,18.32
1,High,less,22.01,19.27,14.13
2,Less,high,27.71,17.12,21.49
3,Less,less,22.36,18.07,12.86


#### Interpretation

        -> Equally distributed between HIGH/LESS affluence classification. 
        -> Dip in fe2rr

In [35]:
df_v4 = df_analysis_v1.groupby(['flag_fe_density', 'ocara_tag_1']).agg({ 'pickup_hex_8' : 'sum',
                                                              'hex_distribution': 'sum', 
                                                                    'fe_count' : 'sum',
                                                                    'requested_orders' : 'sum',
                                                                    'ocara' : 'sum'
                                                                   }).reset_index()

df_v4['fe2rr'] = (df_v4['requested_orders']*100/df_v4['fe_count']).round(2)
df_v4['ocara_%'] = (df_v4['ocara']*100/df_v4['requested_orders']).round(2)

df_v4[['flag_fe_density', 'ocara_tag_1', 'hex_distribution', 'fe2rr','ocara_%']]

## FE Density x OCARA

,flag_fe_density,ocara_tag_1,hex_distribution,fe2rr,ocara_%
0,High fe,high,14.11,16.68,18.43
1,High fe,less,10.96,19.27,14.16
2,Less fe,high,41.51,18.11,20.60
3,Less fe,less,33.41,18.66,12.96


#### Interpretation
 
        -> Ocara % generally gets increased when fe density is lesser.
        -> fe density wasn't sufficient to be proxy  


In [36]:
df_v3 = df_analysis_v1.groupby(['affluence_hi_tag', 'flag_fe_density', 'ocara_tag_1']).agg({ 'pickup_hex_8' : 'sum',
                                                              'hex_distribution': 'sum', 
                                                                    'fe_count' : 'sum',
                                                                    'requested_orders' : 'sum',
                                                                    'ocara' : 'sum'
                                                                   }).reset_index()

df_v3['fe2rr'] = (df_v3['requested_orders']*100/df_v3['fe_count']).round(2)
df_v3['ocara_%'] = (df_v3['ocara']*100/df_v3['requested_orders']).round(2)

df_v3[['affluence_hi_tag','flag_fe_density','ocara_tag_1', 'pickup_hex_8', 'hex_distribution','fe2rr','ocara_%']]

,affluence_hi_tag,flag_fe_density,ocara_tag_1,pickup_hex_8,hex_distribution,fe2rr,ocara_%
0,High,High fe,high,244,12.21,16.71,18.13
1,High,High fe,less,214,10.71,19.26,14.15
2,High,Less fe,high,314,15.70,18.19,19.58
3,High,Less fe,less,226,11.30,19.49,13.67
4,Less,High fe,high,38,1.90,16.36,21.02
5,Less,High fe,less,5,0.25,20.52,15.57
6,Less,Less fe,high,516,25.81,18.00,21.98
7,Less,Less fe,less,442,22.11,16.70,11.00


In [37]:
print(round(38*100.00/45, 2))

84.44


## Summary
    
        
        -> 55 % Hex are identified to be having high ocara % (Higher than mean/median/Q2 hex level ocara threshold)
        
        -> Equally distributed between HIGH/LESS affluence classification. 
                        
        -> Ocara % is high when it's identified less affluence hex.

        -> 84 % of our less affluence experimental hex's are identified to be having high ocara % threshold.
        

In [39]:
city = 'Bangalore'
service = 'Link'
start_date = '20230717'
end_date = '20230806'

## New Bad Match logic 


        Good Match : FM < 0.5 * LM
        Okay Match : 0.5 * LM <= FM <= 2*LM
        Bad Match : FM > 2 * LM

In [65]:
## Datasets

raw_dataset = f"""

    with base_data as (

        select
            city_name as City,
            accept_to_pickup_distance as First_Mile,
            ride_distance as Last_Mile,
            pickup_cluster as Pickup_Cluster,
            pickup_location_hex_8,
            drop_cluster as Drop_Cluster,
            order_id as Order_ID,
            captain_id as Captain_ID,
            customer_id as Customer_ID,
            cancel_reason as Cancel_Reason,
            order_status as Order_Status,
            service_obj_service_name as Service_Type,
            service_detail_id

        from
            orders.order_logs_snapshot

        where
            yyyymmdd >= '{start_date}'
            AND yyyymmdd <= '{end_date}'
            AND (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and service_detail_id = '57370b61a6855d70057417d1'
            and accept_to_pickup_distance is not null
            and distance_final_distance is not null
        )

    select

        pickup_location_hex_8,
        Gross_Orders,
        OCARA_Orders,
        Net_Orders,
        BM_Gross_Orders,
        BM_OCARA_Orders,
        BM_Net_Orders

    from
    (
        select
            pickup_location_hex_8,
            count(distinct Order_ID) as Gross_Orders,
            count(distinct Order_ID) filter (where (Order_Status = 'customerCancelled') and (Cancel_Reason not in ('Cancellation because of nudged order booking','order cancelled before rider accepted', 'Order cancelled before rider was mapped', ''))) as OCARA_Orders,
            count(distinct Order_ID) filter (where Order_Status = 'dropped') as Net_Orders,
            count(distinct Order_ID) filter (where (First_Mile is not null) and (Last_Mile is not null) and (First_Mile > 2 * Last_Mile)) as BM_Gross_Orders,
            count(distinct Order_ID) filter (where (Order_Status = 'customerCancelled') and (Cancel_Reason not in ('Cancellation because of nudged order booking','order cancelled before rider accepted', 'Order cancelled before rider was mapped', '')) and (First_Mile > 2 * Last_Mile)) as BM_OCARA_Orders,
            count(distinct Order_ID) filter (where (Order_Status = 'dropped') and (First_Mile > 2 * Last_Mile)) as BM_Net_Orders
        from
            base_data
        group by 1
    )

    order by 1
"""
df_raw_dataset = pd.read_sql(raw_dataset, connection)
df_raw_dataset.head(3)

,pickup_location_hex_8,Gross_Orders,OCARA_Orders,Net_Orders,BM_Gross_Orders,BM_OCARA_Orders,BM_Net_Orders
0,8860144059fffff,1,1,0,0,0,0
1,886014413dfffff,1,1,0,0,0,0
2,8860144145fffff,1,0,0,0,0,0


In [66]:
df_raw_dataset.to_csv('/Users/rapido/local-datasets/affluence/raw/raw_data_bad_mismatch_ocara_{}_{}_{}_to_{}.csv' \
                                .format(city, service,start_date,end_date)
                               , index = False)

In [67]:
raw_data_bad_mismatch_ocara = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_data_bad_mismatch_ocara_{}_{}_{}_to_{}.csv' \
                                .format(city, service,start_date,end_date))

In [68]:
raw_data_bad_mismatch_ocara

,pickup_location_hex_8,Gross_Orders,OCARA_Orders,Net_Orders,BM_Gross_Orders,BM_OCARA_Orders,BM_Net_Orders
0,8860144059fffff,1,1,0,0,0,0
1,886014413dfffff,1,1,0,0,0,0
2,8860144145fffff,1,0,0,0,0,0
3,886014414dfffff,2,0,1,0,0,0
4,8860144159fffff,6,1,5,0,0,0
...,...,...,...,...,...,...,...
1980,8861893717fffff,2,1,0,0,0,0
1981,8861893733fffff,1,1,0,0,0,0
1982,8861893737fffff,3,1,2,0,0,0
1983,8861893893fffff,1,0,0,0,0,0


In [69]:
affluence_hex_mapping_list = pd.read_csv('/Users/rapido/local-datasets/affluence/affluence_hex_mapping_list.csv')

In [70]:
affluence_hex_mapping_list

,city,pickup_cluster,pickup_hex_8,affluence_hi_tag
0,Bangalore,Hoskote_BLR,8861893893fffff,Less
1,Bangalore,Attibele,8861893713fffff,Less
2,Bangalore,Attibele,88618936a7fffff,Less
3,Bangalore,Sarjapur,88618935ebfffff,High
4,Bangalore,Sarjapur,88618935e9fffff,Less
...,...,...,...,...
1994,Bangalore,Bidadi,886014413dfffff,High
1995,Bangalore,Bidadi,8860144135fffff,High
1996,Bangalore,Bidadi,8860144125fffff,Less
1997,Bangalore,Bidadi,8860144113fffff,Less


In [71]:
df_merge = pd.merge(raw_data_bad_mismatch_ocara,
                    affluence_hex_mapping_list,
                    how = 'inner',
                    left_on = ['pickup_location_hex_8'],
                    right_on = ['pickup_hex_8']
                    )
df_merge

,pickup_location_hex_8,Gross_Orders,OCARA_Orders,Net_Orders,BM_Gross_Orders,BM_OCARA_Orders,BM_Net_Orders,city,pickup_cluster,pickup_hex_8,affluence_hi_tag
0,886014413dfffff,1,1,0,0,0,0,Bangalore,Bidadi,886014413dfffff,High
1,8860144145fffff,1,0,0,0,0,0,Bangalore,Bidadi,8860144145fffff,High
2,886014414dfffff,2,0,1,0,0,0,Bangalore,Bidadi,886014414dfffff,Less
3,8860144159fffff,6,1,5,0,0,0,Bangalore,Bidadi,8860144159fffff,Less
4,886014415dfffff,2,1,0,0,0,0,Bangalore,Bidadi,886014415dfffff,Less
...,...,...,...,...,...,...,...,...,...,...,...
1819,88618935e7fffff,20,6,13,0,0,0,Bangalore,Sarjapur,88618935e7fffff,Less
1820,88618935e9fffff,1,0,1,0,0,0,Bangalore,Sarjapur,88618935e9fffff,Less
1821,88618935ebfffff,6,2,4,0,0,0,Bangalore,Sarjapur,88618935ebfffff,High
1822,8861893713fffff,22,7,10,0,0,0,Bangalore,Attibele,8861893713fffff,Less


In [75]:
df_merge_funnel = df_merge.groupby(['affluence_hi_tag']).agg({
                                            'Gross_Orders' : 'sum',
                                            'OCARA_Orders' : 'sum',
                                            'Net_Orders' : 'sum',
                                            'BM_Gross_Orders' : 'sum',
                                            'BM_OCARA_Orders' : 'sum',
                                            'BM_Net_Orders' : 'sum'
                                            }).reset_index()
df_merge_funnel

,affluence_hi_tag,Gross_Orders,OCARA_Orders,Net_Orders,BM_Gross_Orders,BM_OCARA_Orders,BM_Net_Orders
0,High,2105736,493024,1514866,9114,4811,3364
1,Less,272964,79440,178406,1820,983,556


In [87]:
df_merge_funnel['Overall OCARA %'] = (df_merge_funnel['OCARA_Orders']*100.0/df_merge_funnel['Gross_Orders']).round(2)
df_merge_funnel['BM OCARA %'] = (df_merge_funnel['BM_OCARA_Orders']*100.0/df_merge_funnel['BM_Gross_Orders']).round(2)
# df_merge_funnel['BM G2N %'] = (df_merge_funnel['BM_Net_Orders']*100.0/df_merge_funnel['BM_Gross_Orders']).round(2)


df_merge_funnel['BM Gross Orders %'] = (df_merge_funnel['BM_Gross_Orders']*100.0/df_merge_funnel['Gross_Orders']).round(2)
df_merge_funnel['BM OCARA Orders %'] = (df_merge_funnel['BM_OCARA_Orders']*100.0/df_merge_funnel['OCARA_Orders']).round(2)
# df_merge_funnel['BM Net Orders %'] = (df_merge_funnel['BM_Net_Orders']*100.0/df_merge_funnel['Net_Orders']).round(2)

df_merge_funnel.rename(columns = {'affluence_hi_tag' : 'affluence'}, inplace = True)
df_merge_funnel

,affluence,Gross_Orders,OCARA_Orders,Net_Orders,BM_Gross_Orders,BM_OCARA_Orders,BM_Net_Orders,Overall OCARA %,BM OCARA %,BM G2N %,BM Gross Orders %,BM OCARA Orders %,BM Net Orders %
0,High,2105736,493024,1514866,9114,4811,3364,23.41,52.79,36.91,0.43,0.98,0.22
1,Less,272964,79440,178406,1820,983,556,29.10,54.01,30.55,0.67,1.24,0.31


In [89]:
df_merge_funnel[['affluence', 'Gross_Orders', 'OCARA_Orders', 'BM_Gross_Orders', 'BM_OCARA_Orders',
       'Overall OCARA %', 'BM OCARA %', 'BM Gross Orders %', 'BM OCARA Orders %']]

,affluence,Gross_Orders,OCARA_Orders,BM_Gross_Orders,BM_OCARA_Orders,Overall OCARA %,BM OCARA %,BM Gross Orders %,BM OCARA Orders %
0,High,2105736,493024,9114,4811,23.41,52.79,0.43,0.98
1,Less,272964,79440,1820,983,29.10,54.01,0.67,1.24
